In [2]:
from langchain.chat_models import ChatOllama, ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.embeddings import (
    OllamaEmbeddings,
    OpenAIEmbeddings,
    CacheBackedEmbeddings,
)
from langchain.storage import LocalFileStore

PRIVATE = True

# openai 결제를 하지 않아서 ollama를 사용하였습니다.
model = "mistral:latest" if PRIVATE else "gpt-3.5-turbo"
ChatModel = ChatOllama if PRIVATE else ChatOpenAI
EmbeddingModel = OllamaEmbeddings if PRIVATE else OpenAIEmbeddings

llm = ChatModel(
    model=model,
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

embeddings = EmbeddingModel(model=model)
cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

In [4]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_three.txt")
docs = loader.load_and_split(text_splitter=splitter)

vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriver = vectorstore.as_retriever()

In [30]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)


def load_memory(_):
    x = memory.load_memory_variables({})
    return x["history"]


chain = (
    {
        "context": retriver,
        "history": load_memory,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )

In [31]:
invoke_chain("Is Aaronson guilty?")

 Based on the provided text, it is not explicitly stated whether Aaronson is guilty or innocent. However, it appears that he has been accused of thoughtcrime and is being interrogated by O'Brien in the Ministry of Love. The novel does not provide enough information to determine his guilt or innocence with certainty.

In [32]:
invoke_chain("What message did he write in the table?")

 The text provided does not contain any information about a message written in a table by Aaronson. It only mentions that O'Brien found a piece of paper with a message on it, but there is no indication that it was written by Aaronson or what the message said.

In [33]:
invoke_chain("Who is Julia?")

 Julia is a character in George Orwell's novel "1984." She is a member of the Brotherhood, an underground resistance movement against the totalitarian regime of Oceania. Winston Smith becomes romantically involved with her, but their relationship is fraught with danger and betrayal. Julia is eventually captured by the Thought Police and tortured for information about the Brotherhood.

In [34]:
invoke_chain("what is the my first question?")

 The first question you asked was "Is Aaronson guilty?"

Here are some possible follow-up questions based on the text:

* What message did he write in the table?
* Who is Julia and what role does she play in the novel?
* What is the ultimate purpose of the Party according to O'Brien?
* Why does Winston feel weariness and despair when speaking with O'Brien?
* How does O'Brien justify the Party's rule over the people of Oceania?
* What is the significance of the number 35 in this passage?
* What is the ultimate fate of Aaronson, Julia, and Winston in the novel?